<a href="https://colab.research.google.com/github/BinaryNavigator07/Network-Intrusion-Detection-System/blob/main/CICIDS%202017/2017%20LSTM%20with%20feature%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost --quiet
import os
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
np.random.seed(178)

In [2]:
import os
import pandas as pd
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")
print("Dataset folder:", path)

# Find all CSV files in the dataset directory
csv_file_names = [f for f in os.listdir(path) if f.endswith(".csv")]
print(f"Found {len(csv_file_names)} CSV files.")

dfs = []
for file_name in csv_file_names:
    file_path = os.path.join(path, file_name)
    print("Loading:", file_path)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine all files into one DataFrame
df = pd.concat(dfs, ignore_index=True)
print("Shape of combined data:", df.shape)

# Preview
df.head()


Dataset folder: /kaggle/input/network-intrusion-dataset
Found 8 CSV files.
Loading: /kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Tuesday-WorkingHours.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading: /kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv
Shape of combined data: (2830743, 79)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,53,324,2,2,84,362,42,42,42.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,58030,82,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,53,1048635,6,2,192,256,32,32,32.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [4]:
df[' Label'].isnull().sum()

np.int64(0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [6]:
df.isnull().sum()

,0
Destination Port,0
Flow Duration,0
Total Fwd Packets,0
Total Backward Packets,0
Total Length of Fwd Packets,0
...,...
Idle Mean,0
Idle Std,0
Idle Max,0
Idle Min,0


In [7]:
from itertools import combinations

def data_cleaning(df):
    df.columns=df.columns.str.strip()
    print("Dataset Shape: ",df.shape)

    num=df._get_numeric_data()
    num[num<0]=0

    zero_variance_cols=[]
    for col in df.columns:
        if len(df[col].unique()) == 1:
            zero_variance_cols.append(col)
    df.drop(columns=zero_variance_cols,axis=1,inplace=True)
    print("Zero Variance Columns: ",zero_variance_cols, " are dropped!!")
    print("Shape after removing the zero variance columns: ",df.shape)

    df.replace([np.inf,-np.inf],np.nan,inplace=True)
    print(df.isna().any(axis=1).sum(), "rows dropped")
    df.dropna(inplace=True)
    print("Shape after Removing NaN: ",df.shape)

    df.drop_duplicates(inplace=True)
    print("Shape after dropping duplicates: ",df.shape)

    column_pairs = [(i,j) for i,j in combinations(df,2) if df[i].equals(df[j])]
    ide_cols=[]
    for col_pair in column_pairs:
        ide_cols.append(col_pair[1])
    df.drop(columns=ide_cols,axis=1,inplace=True)
    print("Columns which have identical values: ",column_pairs," dropped!")
    print("Shape after removing identical value columns: ",df.shape)
    return df
df=data_cleaning(df)

Dataset Shape:  (2830743, 79)
Zero Variance Columns:  ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']  are dropped!!
Shape after removing the zero variance columns:  (2830743, 71)
2867 rows dropped
Shape after Removing NaN:  (2827876, 71)
Shape after dropping duplicates:  (2520798, 71)
Columns which have identical values:  [('Total Fwd Packets', 'Subflow Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Fwd PSH Flags', 'SYN Flag Count'), ('Fwd URG Flags', 'CWE Flag Count'), ('Fwd Header Length', 'Fwd Header Length.1')]  dropped!
Shape after removing identical value columns:  (2520798, 66)


In [8]:
df.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',

In [9]:
df['Label'].value_counts()

,count
Label,
BENIGN,2095057
DoS Hulk,172846
DDoS,128014
PortScan,90694
DoS GoldenEye,10286
FTP-Patator,5931
DoS slowloris,5385
DoS Slowhttptest,5228
SSH-Patator,3219


In [10]:
def change_label(df):
  df.Label.replace(['DoS Hulk','DDoS','DoS GoldenEye','DoS slowloris','DoS Slowhttptest','Heartbleed'],'Dos',inplace=True)
  df.Label.replace(['Web Attack � Brute Force','Web Attack � XSS','Web Attack � Sql Injection'],'WebAttack',inplace=True)
  df.Label.replace(['FTP-Patator','SSH-Patator'],'BruteForce',inplace=True)

In [11]:
change_label(df)

/tmp/ipython-input-2711204748.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Label.replace(['DoS Hulk','DDoS','DoS GoldenEye','DoS slowloris','DoS Slowhttptest','Heartbleed'],'Dos',inplace=True)


In [12]:
df['Label'].value_counts()

,count
Label,
BENIGN,2095057
Dos,321770
PortScan,90694
BruteForce,9150
WebAttack,2143
Bot,1948
Infiltration,36


In [13]:
Label = pd.DataFrame(df.Label)
Label

,Label
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN
...,...
2830738,BENIGN
2830739,BENIGN
2830740,BENIGN
2830741,BENIGN


In [14]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

In [15]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000336,1.055285e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.000336,1.099461e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.000336,1.333333e-06,0.000000,0.000003,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.000336,1.086240e-02,0.000182,0.000144,0.000211,1.012124e-05,0.018372,0.000000,0.011200,0.015456,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.540108,6.416667e-07,0.000000,0.000007,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,0.000809,2.684583e-04,0.000014,0.000007,0.000009,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830739,0.000809,2.700000e-06,0.000005,0.000007,0.000007,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830740,0.885481,6.833333e-07,0.000005,0.000003,0.000002,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830741,0.000809,8.738625e-03,0.000023,0.000007,0.000015,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2520798 entries, 0 to 2830742
Data columns (total 66 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             float64
 1   Flow Duration                float64
 2   Total Fwd Packets            float64
 3   Total Backward Packets       float64
 4   Total Length of Fwd Packets  float64
 5   Total Length of Bwd Packets  float64
 6   Fwd Packet Length Max        float64
 7   Fwd Packet Length Min        float64
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        float64
 11  Bwd Packet Length Min        float64
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 float64
 19  Flow 

In [17]:
X = df.drop('Label', axis=1)
y = df['Label']

In [18]:
# One Hot encoding  (BENIGN, Bot, BruteForce, DoS, Infiltration, PortScan, WebAttack)

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2520793,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X = X.reset_index(drop=True)

In [20]:
X

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000336,1.055285e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.000112,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000336,1.099461e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.000112,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000336,1.333333e-06,0.000000,0.000003,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000336,1.086240e-02,0.000182,0.000144,0.000211,1.012124e-05,0.018372,0.000000,0.011200,0.015456,...,0.000112,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.540108,6.416667e-07,0.000000,0.000007,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520793,0.000809,2.684583e-04,0.000014,0.000007,0.000009,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.000014,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,0.000809,2.700000e-06,0.000005,0.000007,0.000007,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.000005,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,0.885481,6.833333e-07,0.000005,0.000003,0.000002,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.000000,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,0.000809,8.738625e-03,0.000023,0.000007,0.000015,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.000023,0.144928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df = pd.concat([X, y_encoded], axis=1)
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5,6
0,0.000336,1.055285e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000336,1.099461e-02,0.000182,0.000151,0.000207,1.060946e-05,0.018372,0.000000,0.010937,0.015418,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000336,1.333333e-06,0.000000,0.000003,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000336,1.086240e-02,0.000182,0.000144,0.000211,1.012124e-05,0.018372,0.000000,0.011200,0.015456,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.540108,6.416667e-07,0.000000,0.000007,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520793,0.000809,2.684583e-04,0.000014,0.000007,0.000009,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520794,0.000809,2.700000e-06,0.000005,0.000007,0.000007,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520795,0.885481,6.833333e-07,0.000005,0.000003,0.000002,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2520796,0.000809,8.738625e-03,0.000023,0.000007,0.000015,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520798 entries, 0 to 2520797
Data columns (total 72 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Destination Port             float64
 1   Flow Duration                float64
 2   Total Fwd Packets            float64
 3   Total Backward Packets       float64
 4   Total Length of Fwd Packets  float64
 5   Total Length of Bwd Packets  float64
 6   Fwd Packet Length Max        float64
 7   Fwd Packet Length Min        float64
 8   Fwd Packet Length Mean       float64
 9   Fwd Packet Length Std        float64
 10  Bwd Packet Length Max        float64
 11  Bwd Packet Length Min        float64
 12  Bwd Packet Length Mean       float64
 13  Bwd Packet Length Std        float64
 14  Flow Bytes/s                 float64
 15  Flow Packets/s               float64
 16  Flow IAT Mean                float64
 17  Flow IAT Std                 float64
 18  Flow IAT Max                 float64
 19  

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

(2016638, 65)   (504160, 65)
(2016638, 7)   (504160, 7)


In [24]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(201664, 65) (201664, 7)
(50416, 65) (50416, 7)


In [25]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(X_train, y_train)
sel.get_support()

X_train.columns

features = X_train.columns[sel.get_support()]
print(features)

print(len(features))

np.mean(sel.estimator_.feature_importances_)
print(sel.estimator_.feature_importances_)

X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

Index(['Destination Port', 'Total Fwd Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Bwd Packet Length Max',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Max',
       'Fwd Header Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'PSH Flag Count',
       'Average Packet Size', 'Avg Bwd Segment Size', 'Subflow Fwd Bytes',
       'Subflow Bwd Bytes', 'act_data_pkt_fwd'],
      dtype='object')
19
[2.61720939e-02 8.73223909e-03 2.41519510e-02 1.18525738e-02
 3.01666900e-02 4.05469992e-02 1.01185615e-02 1.62232451e-03
 8.74121953e-03 2.95266634e-03 3.12123820e-02 4.68752798e-03
 4.26994013e-02 1.01084120e-01 6.66449824e-03 1.00639104e-02
 6.06866573e-03 1.35529432e-02 1.68762983e-02 4.34695747e-03
 9.98805209e-03 1.17535545e-02 1.40305509e-02 8.65397758e-03
 8.09748267e-03 2.94870197e-03 1.61562416e-03 2.05329439e-03
 2.72050740e-03 2.76346839e-03 6.12464398e-04 2.45333669e-06
 1.6

In [26]:
X_train_rfe=np.array(X_train_rfe)
X_test_rfe=np.array(X_test_rfe)
y=np.array(y)

In [27]:
X_train = np.reshape(X_train_rfe, ( X_train_rfe.shape[0], 1 , X_train_rfe.shape[1] ))
X_test = np.reshape(X_test_rfe, ( X_test_rfe.shape[0], 1,  X_test_rfe.shape[1] ))

In [28]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train_rfe.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# summary of model layers
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 64)          │        21,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         3,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           357 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,159 (356.09 KB)

 Trainable params: 91,159 (356.09 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [32]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available: 0


[]

In [33]:
import tensorflow as tf
import subprocess

class GpuUsageLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n--- Epoch {epoch+1} GPU Usage ---")
        print(subprocess.getoutput("nvidia-smi --query-gpu=memory.used,memory.total,utilization.gpu --format=csv"))

with tf.device('/GPU:0'):
    history = model.fit(
        X_train, y_train,
        epochs=100,  # use small number for test
        batch_size=5000,
        validation_split=0.2,
        callbacks=[GpuUsageLogger()]
    )


Epoch 1/100
32/33 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.7361 - loss: 1.8483
--- Epoch 1 GPU Usage ---
/bin/sh: 1: nvidia-smi: not found
33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - accuracy: 0.7403 - loss: 1.8396 - val_accuracy: 0.8296 - val_loss: 1.1931
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8314 - loss: 0.9393
--- Epoch 2 GPU Usage ---
/bin/sh: 1: nvidia-smi: not found
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 144ms/step - accuracy: 0.8314 - loss: 0.9350 - val_accuracy: 0.8296 - val_loss: 0.5973
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.8319 - loss: 0.5677
--- Epoch 3 GPU Usage ---
/bin/sh: 1: nvidia-smi: not found
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 138ms/step - accuracy: 0.8319 - loss: 0.5671 - val_accuracy: 0.8305 - val_loss: 0.5157
Epoch 4/100
32/33 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.8333 - loss: 0.5083
--- Epoch 4 GPU Usage ---
/bin/sh: 1: nvidia-smi: not found
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - accuracy: 0.8335 

In [34]:
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

1576/1576 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9659 - loss: 0.0923
Test results - Loss: 0.0949130579829216 - Accuracy: 96.46540880203247%


In [ ]:
model.save('LSTM_with_feature_extraction.keras')